In [3]:
import warnings
warnings.filterwarnings('ignore')
    
import pandas as pd
import numpy as np
import datetime
price_raw_1 = pd.read_csv('dataset/minuteByminutePrice/priceDataJanuary2019.csv')
price_raw_2 = pd.read_csv('dataset/minuteByminutePrice/priceDataFebMarch2019.csv')
price_raw = price_raw_1.append(price_raw_2)

news_raw_1 = pd.read_csv('dataset/news_data/jan_score.csv')
news_raw_2 = pd.read_csv('dataset/news_data/feb_score.csv')
news_raw_3 = pd.read_csv('dataset/news_data/march_score.csv')
news_raw = news_raw_1.append(news_raw_2).append(news_raw_3)
news_raw[news_raw.columns[1:]].to_csv('dataset/news_data/jan-feb-mar.csv')
news_raw[news_raw.columns[1:]].head()

,web_url,content,headline,document_type,pub_date,source,section_name,author,word_count,content_no_stopwords,SENTIMENT_VALUE,SENTIMENT,relevance,score
0,https://www.nytimes.com/2019/01/02/world/europ...,"halmstad, sweden — maria hussein, who escaped ...",Where Doulas Calm Nerves and Bridge Cultures D...,article,2019-01-02T22:17:38+0000,The New York Times,World,CHRISTINA ANDERSON,1326,"halmstad, sweden — maria hussein, escaped war ...",1.0,Very Positive,0.002,0.0020
1,https://www.nytimes.com/2019/01/02/us/politics...,"voting rights and partisan gerrymandering, tra...",Voting Issues and Gerrymanders Are Now Key Pol...,article,2019-01-02T17:00:47+0000,The New York Times,U.S.,TRIP GABRIEL,1779,"voting rights partisan gerrymandering, traditi...",1.0,Very Positive,0.060,0.0600
2,https://www.nytimes.com/2019/01/02/world/asia/...,"new delhi — two women, accompanied by plainclo...","2 Indian Women Enter Sabarimala Temple, Settin...",article,2019-01-02T12:37:37+0000,The New York Times,World,KAI SCHULTZ and AYESHA VENKATARAMAN,778,"new delhi — two women, accompanied plainclothe...",-0.8,Very Negative,0.002,-0.0016
3,https://www.nytimes.com/2019/01/01/us/politics...,good tuesday morning and happy new year. here ...,On Politics: Elizabeth Warren Is Running for P...,article,2019-01-01T05:18:48+0000,The New York Times,U.S.,NaN,419,good tuesday morning happy new year. stories m...,0.9,Very Positive,0.060,0.0540
4,https://www.nytimes.com/2018/12/31/world/asia/...,"tokyo — kim jong-un, north korea’s leader, sai...","Kim Jong-un, Ready to Meet Trump ‘at Any Time,...",article,2019-01-01T01:44:15+0000,The New York Times,World,MOTOKO RICH and DAVID E. SANGER,1319,"tokyo — kim jong-un, north korea’s leader, sai...",1.0,Very Positive,0.060,0.0600


In [4]:
# Convert news to minute by minute dataset
news_data = news_raw[['content','headline','document_type','source','SENTIMENT_VALUE','relevance','pub_date','score']]
news_data['pub_date'] =  pd.to_datetime(news_data['pub_date'], format='%Y-%m-%d %H:%M:%S')

def round_timestamp(t):
    t = datetime.datetime(t.year, t.month, t.day, t.hour, t.minute, 0)  
    return t

news_data['timestamp_rounded'] = news_data['pub_date'].apply(round_timestamp)
news_data.to_csv('dataset/to_visualize/jan-feb-mar.csv')


news_data.head()

,content,headline,document_type,source,SENTIMENT_VALUE,relevance,pub_date,score,timestamp_rounded
0,"halmstad, sweden — maria hussein, who escaped ...",Where Doulas Calm Nerves and Bridge Cultures D...,article,The New York Times,1.0,0.002,2019-01-02 22:17:38,0.0020,2019-01-02 22:17:00
1,"voting rights and partisan gerrymandering, tra...",Voting Issues and Gerrymanders Are Now Key Pol...,article,The New York Times,1.0,0.060,2019-01-02 17:00:47,0.0600,2019-01-02 17:00:00
2,"new delhi — two women, accompanied by plainclo...","2 Indian Women Enter Sabarimala Temple, Settin...",article,The New York Times,-0.8,0.002,2019-01-02 12:37:37,-0.0016,2019-01-02 12:37:00
3,good tuesday morning and happy new year. here ...,On Politics: Elizabeth Warren Is Running for P...,article,The New York Times,0.9,0.060,2019-01-01 05:18:48,0.0540,2019-01-01 05:18:00
4,"tokyo — kim jong-un, north korea’s leader, sai...","Kim Jong-un, Ready to Meet Trump ‘at Any Time,...",article,The New York Times,1.0,0.060,2019-01-01 01:44:15,0.0600,2019-01-01 01:44:00


In [3]:
# Aggregate news_data
# news_data['value'] = news_data['SENTIMENT_VALUE'] *  news_data['relevance'] 
# news_data.groupby('pub_date_rounded').agg({'SENTIMENT_VALUE':['sum', 'max','count'], 
#                          'relevance': lambda x: x.max() - x.min()})
# 2019-01-01 10:00:00	
news_data_aggregated = news_data.groupby('timestamp_rounded').agg({'score':['sum', 'count']})
news_data_aggregated.head(10)

score      
                        sum count
timestamp_rounded                
2019-01-01 00:36:00 -0.0120     1
2019-01-01 00:45:00 -0.0600     1
2019-01-01 01:04:00 -0.0018     1
2019-01-01 01:44:00  0.0600     1
2019-01-01 05:18:00  0.0540     1
2019-01-01 08:24:00 -0.0014     1
2019-01-01 10:00:00  0.1180     3
2019-01-01 13:37:00 -0.0020     1
2019-01-01 15:28:00  0.0600     1
2019-01-01 15:43:00 -0.0540     1

In [4]:
# process price_data
price_raw['timestamp'] = price_raw['Date'] + ' ' + price_raw['Time']
price_raw['timestamp'] =  pd.to_datetime(price_raw['timestamp'], format='%Y-%m-%d %H:%M:%S')
price_raw['timestamp_rounded'] = price_raw['timestamp'].apply(round_timestamp)

price_raw.head()

,Close,Date,High,Low,Open,Time,Unnamed: 0,Volume,id,timestamp,timestamp_rounded
0,3698.00,2019-01-01,3698.79,3693.85,3693.85,00:00:00,NaN,5.491289,3676576.0,2019-01-01 00:00:00,2019-01-01 00:00:00
1,3690.65,2019-01-01,3694.72,3690.65,3694.72,00:01:00,NaN,9.500151,3676577.0,2019-01-01 00:01:00,2019-01-01 00:01:00
2,3686.62,2019-01-01,3689.73,3686.62,3689.73,00:02:00,NaN,0.965966,3676578.0,2019-01-01 00:02:00,2019-01-01 00:02:00
3,3692.35,2019-01-01,3692.85,3688.32,3692.85,00:03:00,NaN,0.296662,3676579.0,2019-01-01 00:03:00,2019-01-01 00:03:00
4,3690.34,2019-01-01,3692.35,3690.34,3692.35,00:04:00,NaN,0.111622,3676580.0,2019-01-01 00:04:00,2019-01-01 00:04:00


In [5]:
price_data_aggregated = price_raw.groupby('timestamp_rounded').agg({
    'Open':['mean'],
    'High':['max'],
    'Low':['min'],
    'Close':['mean'],
    'Volume':['sum'] 
})

price_raw.count()

Close                92623
Date                 92623
High                 92623
Low                  92623
Open                 92623
Time                 92623
Unnamed: 0           52481
Volume               92623
id                   40142
timestamp            92623
timestamp_rounded    92623
dtype: int64

In [6]:
final_data = price_data_aggregated.join(news_data_aggregated, how='outer', on = ['timestamp_rounded'])
final_data.count()
final_data = final_data.reset_index()
final_data.columns = ['Timestamp_Rounded', 'Open','High','Low','Close','Volume','Score','news_count']
# Fill null values for news with 0 (neutrial news ?!)
final_data['news_count'] = final_data['news_count'].fillna(0)
final_data['Score'] = final_data['Score'].fillna(0)
final_data

,Timestamp_Rounded,Open,High,Low,Close,Volume,Score,news_count
0,2019-01-01 00:00:00,3693.85,3698.79,3693.85,3698.00,5.491289,0.0000,0.0
1,2019-01-01 00:01:00,3694.72,3694.72,3690.65,3690.65,9.500151,0.0000,0.0
2,2019-01-01 00:02:00,3689.73,3689.73,3686.62,3686.62,0.965966,0.0000,0.0
3,2019-01-01 00:03:00,3692.85,3692.85,3688.32,3692.35,0.296662,0.0000,0.0
4,2019-01-01 00:04:00,3692.35,3692.35,3690.34,3690.34,0.111622,0.0000,0.0
5,2019-01-01 00:05:00,3690.40,3690.85,3690.40,3690.85,2.247676,0.0000,0.0
6,2019-01-01 00:06:00,3688.61,3688.61,3687.08,3687.08,1.556315,0.0000,0.0
7,2019-01-01 00:07:00,3684.59,3685.52,3684.59,3684.77,0.823009,0.0000,0.0
8,2019-01-01 00:08:00,3684.97,3688.79,3683.45,3683.45,4.493121,0.0000,0.0
9,2019-01-01 00:09:00,3685.00,3685.12,3684.58,3685.12,1.077187,0.0000,0.0


In [7]:
final_data.to_csv('dataset/aggregated/min_by_min/jan-feb-mar.csv')